# Infinite Loop: Pack -> Rotate -> BP
Running a continuous loop of optimization steps to maximize score improvement.

In [ ]:
import os
import time
import subprocess
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar

print(f"Current working directory: {os.getcwd()}")

# Setup paths
DATA_DIR = '/home/data'
START_SUBMISSION = 'submission_start.csv'
CURRENT_SUBMISSION = 'submission.csv'
BEST_SUBMISSION = 'submission_best.csv'

# Ensure binaries are executable
subprocess.run(['chmod', '+x', './tree_packer'], check=True)
subprocess.run(['chmod', '+x', './bp'], check=True)

# ----------------------------------------------------------------
# Geometry & Validation Classes
# ----------------------------------------------------------------
getcontext().prec = 30
scale_factor = 1

class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x).strip('s'))
        self.center_y = Decimal(str(center_y).strip('s'))
        self.angle = Decimal(str(angle).strip('s'))
        
        # Tree definition
        trunk_w = Decimal('0.15'); trunk_h = Decimal('0.2')
        base_w = Decimal('0.7'); mid_w = Decimal('0.4'); top_w = Decimal('0.25')
        tip_y = Decimal('0.8'); tier_1_y = Decimal('0.5'); tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0'); trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x * scale_factor), yoff=float(self.center_y * scale_factor))

    def clone(self):
        return ChristmasTree(self.center_x, self.center_y, self.angle)

def load_configuration_from_df(n: int, df: pd.DataFrame) -> list[ChristmasTree]:
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        trees.append(ChristmasTree(row["x"], row["y"], row["deg"]))
    return trees

def get_score(trees: list[ChristmasTree], n: int) -> float:
    if not trees: return 0.0
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / float(scale_factor) for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    side_length = max(max_x - min_x, max_y - min_y)
    return side_length**2 / n

def has_overlap(trees: list[ChristmasTree]) -> bool:
    if len(trees) <= 1: return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i: continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def calculate_total_score(file_path):
    try:
        df = pd.read_csv(file_path)
        total_score = 0.0
        for n in range(1, 201):
            trees = load_configuration_from_df(n, df)
            if trees:
                total_score += get_score(trees, n)
        return total_score
    except Exception as e:
        print(f"Error calculating score: {e}")
        return float('inf')

# ----------------------------------------------------------------
# Rotation Optimization Logic
# ----------------------------------------------------------------
def calculate_bbox_side_at_angle(angle_deg, points):
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def optimize_rotation_for_group(trees):
    if not trees: return trees, 0.0
    
    all_points = []
    for tree in trees:
        all_points.extend(list(tree.polygon.exterior.coords))
    points_np = np.array(all_points)
    
    try:
        hull_points = points_np[ConvexHull(points_np).vertices]
    except:
        hull_points = points_np 

    initial_side = calculate_bbox_side_at_angle(0, hull_points)

    res = minimize_scalar(lambda a: calculate_bbox_side_at_angle(a, hull_points),
                          bounds=(0.0, 90.0), method='bounded')
    found_angle_deg = res.x
    found_side = res.fun

    if found_side < initial_side:
        best_angle_deg = found_angle_deg
    else:
        best_angle_deg = 0.0
        
    if abs(best_angle_deg) > 1e-9:
        min_x, min_y = points_np.min(axis=0)
        max_x, max_y = points_np.max(axis=0)
        rotation_center = np.array([(min_x + max_x) / 2.0, (min_y + max_y) / 2.0])
        
        angle_rad = np.radians(best_angle_deg)
        c, s = np.cos(angle_rad), np.sin(angle_rad)
        rot_matrix = np.array([[c, -s], [s, c]])
        
        new_trees = []
        for tree in trees:
            p = np.array([float(tree.center_x), float(tree.center_y)])
            shifted = p - rotation_center
            rotated_p = shifted.dot(rot_matrix.T) + rotation_center
            new_angle = float(tree.angle) + best_angle_deg
            new_trees.append(ChristmasTree(rotated_p[0], rotated_p[1], new_angle))
        return new_trees, best_angle_deg
    else:
        return trees, 0.0

def run_rotation_optimization(input_csv, output_csv):
    print(f"Running Rotation Optimization on {input_csv}...")
    df = pd.read_csv(input_csv)
    new_rows = []
    for n in range(1, 201):
        trees = load_configuration_from_df(n, df)
        if not trees: continue
        optimized_trees, angle_change = optimize_rotation_for_group(trees)
        for i, tree in enumerate(optimized_trees):
            new_rows.append({
                'id': f'{n:03d}_{i}',
                'x': f's{tree.center_x}',
                'y': f's{tree.center_y}',
                'deg': f's{tree.angle}'
            })
    new_df = pd.DataFrame(new_rows)
    new_df = new_df[['id', 'x', 'y', 'deg']]
    new_df.to_csv(output_csv, index=False)
    print(f"Rotation Optimization complete.")

# ----------------------------------------------------------------
# Main Pipeline
# ----------------------------------------------------------------
class CppOptimizer:
    def __init__(self):
        pass
        
    def run_packer(self, input_file, output_file, iterations=10000, restarts=16):
        print(f"Running tree_packer: n={iterations}, r={restarts}...")
        subprocess.run(['./tree_packer', '-i', input_file, '-o', output_file, '-n', str(iterations), '-r', str(restarts)], check=True)
        
    def run_bp(self, input_file, output_file):
        print("Running Backward Propagation...")
        subprocess.run(['./bp', input_file, output_file], check=True)

# Execution
model = CppOptimizer()

# 1. Initialize
if os.path.exists(START_SUBMISSION):
    print(f"Starting from {START_SUBMISSION}")
    subprocess.run(['cp', START_SUBMISSION, CURRENT_SUBMISSION], check=True)
else:
    print("Starting from sample_submission.csv")
    subprocess.run(['cp', os.path.join(DATA_DIR, 'sample_submission.csv'), CURRENT_SUBMISSION], check=True)

best_score = calculate_total_score(CURRENT_SUBMISSION)
print(f"Initial Score: {best_score}")
subprocess.run(['cp', CURRENT_SUBMISSION, BEST_SUBMISSION], check=True)

start_time = time.time()
MAX_DURATION = 3300 # 55 minutes
loop_count = 0

while time.time() - start_time < MAX_DURATION:
    loop_count += 1
    print(f"\n=== Loop {loop_count} (Elapsed: {time.time() - start_time:.0f}s) ===")
    
    # Step 1: Pack (Exploration)
    model.run_packer(CURRENT_SUBMISSION, 'submission_packed.csv', iterations=10000, restarts=16)
    
    # Step 2: Rotate (Squeeze)
    run_rotation_optimization('submission_packed.csv', 'submission_rotated.csv')
    
    # Step 3: BP (Exploitation)
    model.run_bp('submission_rotated.csv', 'submission_bp.csv')
    
    # Check score
    current_score = calculate_total_score('submission_bp.csv')
    print(f"Loop {loop_count} Score: {current_score}")
    
    if current_score < best_score:
        print(f"✅ New Best Score! {best_score} -> {current_score}")
        best_score = current_score
        subprocess.run(['cp', 'submission_bp.csv', BEST_SUBMISSION], check=True)
        subprocess.run(['cp', 'submission_bp.csv', CURRENT_SUBMISSION], check=True)
    else:
        print(f"❌ No improvement. Best: {best_score}")
        # Keep current submission as the result of BP to continue exploring from this state?
        # Or revert to best?
        # Usually, we want to continue from the new state to explore, even if it's slightly worse (SA logic), 
        # but BP is deterministic and greedy.
        # tree_packer is SA, so it accepts worse moves internally.
        # If we revert, we might get stuck.
        # If we continue, we might drift.
        # Strategy says: "Pack then re-optimizes N-1 (which is now better) and N (to find new candidates for BP)."
        # So we should probably continue from the result of BP.
        subprocess.run(['cp', 'submission_bp.csv', CURRENT_SUBMISSION], check=True)

print(f"\nLoop finished. Best Score: {best_score}")
subprocess.run(['cp', BEST_SUBMISSION, 'submission.csv'], check=True)


In [ ]:
# Validation
def score_and_validate_submission(file_path: str, max_n: int = 200) -> dict:
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        return {"status": "FAILED", "error": "File Not Found"}
    
    total_score = 0.0
    failed_overlap_n = []
    
    print(f"--- Scoring and Validation: {file_path} (N=1 to {max_n}) ---")
    for n in range(1, max_n + 1):
        trees = load_configuration_from_df(n, df)
        if trees:
            current_score = get_score(trees, n)
            total_score += current_score
            if has_overlap(trees):
                failed_overlap_n.append(n)
                print(f"  ❌ N={n:03d}: OVERLAP DETECTED! (Score contribution: {current_score:.12f})")
    
    if failed_overlap_n:
        status = "FAILED (Overlaps)"
    else:
        status = "SUCCESS"
        
    print(f"**Total Submission Score (Σ S²/N):   {total_score:.14f}**")
    return {"status": status, "total_score": total_score, "failed_overlap_n": failed_overlap_n}

result = score_and_validate_submission('submission.csv')
print(f"Final Result: {result}")
